# LandTrendr Implementation For Decadal Averaging of Spectral Indices

### Requirements: 

Python:

* geemap
* ee
* matplotlib
* numpy
* pandas
* oeel ** 

Other:
* Google Earth Engine Account

The LandTrendr algorithm is highly efficient and extensive tool with documentation at this link: https://emapr.github.io/LT-GEE/

This version uses the latest processing efforts of the Landsat TM+ ETM+ and OLI collection 2

** this package allows the user to run GEE modules directly without the pitfalls of translating JS to Python.

In [1]:
import ee
import geemap
import geemap.ml as ml
from ipygee import chart as chart
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymannkendall as mk
import xarray as xr
import os
# Import date class from datetime module
from datetime import datetime as dt
import datetime
import pytz

# GEE Authentication 
 
 ### Paste the Authetication code into the box below if prompted to save token
 
 
 (press enter to save token)


In [2]:
geemap.ee_initialize()

### Inputs and Outputs



In [3]:
p = '..'

version = 'Version_3_20230303'

l = pd.read_csv(f"{p}/Inputs/{version}/GB.csv").ID
display(l)
ls = l.tolist()

print(f'{len(ls)} catchments processed for hydroclimatic variables')

0     17005
1     18001
2     20007
3     21017
4     21023
      ...  
90    79002
91    79004
92     8009
93    93001
94    94001
Name: ID, Length: 95, dtype: int64

95 catchments processed for hydroclimatic variables


### Load the EE package

The landTrendr package is developed to construct timeseries of landsat imagery for the purpose of land cover detection. The base parameters are optimised for deforestation event detection. 

This utilisation of the GEE asset (with apache license i.e. free for use) allows for the latest version of LandTrendr to be used. In contrast to early versions this allows for the utilisation of the Landsat Collection 2 reprocessing effort with improvements in cloud masking capabilities. Primarily we use "ltgee.buildSRcollection"

In [4]:
oeel = geemap.requireJS()

Map = geemap.Map()

ltgee = geemap.requireJS(r'../JS_module/Adapted_LT_v6.js')

ltgee.availability   # This command lists all the functions within the LandTrenr Module


IMPORTANT! Please be advised:
- This version of the Adapted_LT.js modules
  uses some code adapted from the aut/or: @author Justin Braaten (Google) * @author Zhiqiang Yang (USDA Forest Service) * @author Robert Kennedy (Oregon State University)
The latest edits to this code occur: 08/03/2023 for the adaptation efforts by @Mike OHanrahan (TU DELFT MSc research)


{'version': 'string',
 'buildSensorYearCollection': 'function',
 'getSRcollection': 'function',
 'getCombinedSRcollection': 'function',
 'buildSRcollection': 'function',
 'getCollectionIDlist': 'function',
 'countClearViewPixels': 'function',
 'buildClearPixelCountCollection': 'function',
 'removeImages': 'function',
 'LAIcol': 'function',
 'calcIndex': 'function',
 'standardize': 'function',
 'transformSRcollection': 'function',
 'createTrainingImage': 'function',
 'addTerrainBand': 'function',
 'genGCP': 'function',
 'classifier': 'function',
 'classArea': 'function'}

## Initiate With a Shapefile

This notebook assumes the user has a shapefile saved as an asset on their GEE, the assets used in the CATAPUCII project will be made publicly available in the @mohanrahan repository


In [5]:
asset_dir = 'projects/mohanrahan/assets'

catchment_asset = 'CATAPUCII_Catchments/CAMELS_GB_catchment_boundaries'

dataset = 'CAMELS_GB'

col_string  = 'ID'

crs = 'EPSG:27700'

fignum = 0

RGB_VIS = {'bands':['B3','B2','B1'], 'min':0, 'max':1.5e3}


startYear = 1984

endYear = 2022

startDay = '06-20'

endDay = '08-31'

maskThese = ['cloud', 'shadow', 'snow',]

bandList = ["B1", "B2", "B3", "B4", "B5", "B7", 
           'NBR', 'NDMI', 'NDVI', 'NDSI', 'EVI','GNDVI', 
           'TCB', 'TCG', 'TCW', 'TCA', 'NDFI',] 

## The Table Data

- Here the table, a vector of catchments, is loaded from the users' assets in earth engine 

#TODO rewrite to a local .shp

- The area is calculated of each shape and ranked per area, assuming that the largest is the most computationally expensive
- This is done so that we can iterate from smallest to largest, or the opposite, should any memory issues become apparent.

In [6]:
table = ee.FeatureCollection(f"{asset_dir}/{catchment_asset}")

def set_area_km2(feature):
    '''
    Calculate the area of each geometry in square kilometer
    '''
    area = feature.geometry().area().divide(1000*1000)
    setting = feature.set('area_km2', area)
    return setting

def set_area_pixel(feature):
    aoi = feature.geometry()
    area = ee.Image.pixelArea().divide(1e6).clip(aoi).select('area').reduceRegion(**{
        'reducer':ee.Reducer.sum(),
        'geometry':aoi,
        'scale':30,
        'crs':crs,
        'maxPixels':1e13,
        'bestEffort':True,
        }).get('area')
    setting = feature.set('pixel_area', area)
    return setting

def set_id(feature):
    '''
    Set the system ID as a column
    '''
    getting_name = ee.String(feature.get('system:index'))
    setting_id = feature.set({'system_index':getting_name,})
    return setting_id

table_area = table.map(set_area_km2).map(set_id).map(set_area_pixel)

Filtered_Sorted = table_area.filter(ee.Filter.gt('area_km2', 0)).sort('area_km2', False)  # true ranks from smallest to largest

down = geemap.ee_to_pandas(Filtered_Sorted).set_index(['system_index'])

print(down.keys().values)

df1 = down.loc[down['ID'].isin(ls)]

print(f'The length of the dataframe generated from the EE asset {len(df1)}')

sys_index = df1.index.to_list()

display(df1)

#df1.loc[sys_index[-1]]

['pixel_area' 'area_km2' 'SOURCE' 'VERSION' 'ID' 'EXPORTED' 'ID_STRING']
The length of the dataframe generated from the EE asset 95


,pixel_area,area_km2,SOURCE,VERSION,ID,EXPORTED,ID_STRING
system_index,,,,,,,
00000000000000000088,1355.679450,1349.764903,National River Flow Archive,1.3,27071,1518422400000,27071
00000000000000000235,1145.767150,1140.885961,National River Flow Archive,1.3,71001,1518422400000,71001
000000000000000001c0,1125.603290,1121.172318,National River Flow Archive,1.3,54008,1518422400000,54008
00000000000000000207,897.948531,894.478359,National River Flow Archive,1.3,62001,1518422400000,62001
0000000000000000025a,798.157955,794.507522,National River Flow Archive,1.3,79002,1518422400000,79002
...,...,...,...,...,...,...,...
0000000000000000021b,12.768404,12.719149,National River Flow Archive,1.3,67010,1518422400000,67010
00000000000000000082,11.353532,11.304831,National River Flow Archive,1.3,27047,1518422400000,27047
000000000000000001df,10.506362,10.466687,National River Flow Archive,1.3,55008,1518422400000,55008


In [7]:
len1 = len(df1.ID.values)
len2 = len(ls)

if len1 > len2:
    print(f'catchment{ set(df1.station_re.values).symmetric_difference(ls)} is/are missing from the catchment sets')
elif len2 >len1:
    print(f'catchment{ set(df1.station_re.values).symmetric_difference(ls)} is/are missing from the EE asset')
else:
    print('The number of catchments with Hydroclimatic indices calculated match the length filtered EE asset\nThere seems to be no mismatch\nContinue... ')

The number of catchments with Hydroclimatic indices calculated match the length filtered EE asset
There seems to be no mismatch
Continue... 


In [8]:
gdf = geemap.ee_to_pandas(Filtered_Sorted)

if not os.path.exists(f'../Outputs/{dataset}/'):
    os.makedirs(f'../Outputs/{dataset}/')

gdf.to_excel(f'../Outputs/{dataset}/{dataset}_catchment_table.xlsx')

In [9]:
Map = geemap.Map()
# Map.setOptions('TERRAIN')
Map.addLayer(Filtered_Sorted.filter(ee.Filter.inList('ID', ee.List(ls)).Not()), {'color':'red'}, 'red: Not Included')
Map.addLayer(Filtered_Sorted.filter(ee.Filter.inList('ID', ee.List(ls))), {'color': 'green'}, 'green: Included')
Map.centerObject(Filtered_Sorted, 6)


Map

Map(center=[53.49722684216607, -2.3029239738111555], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
def clip_collection(image: ee.Image)-> ee.Image:
    
    """
    reduce the size of the image colelction to be only pixels relevant to the aoi
    """
    return image.clip(aoi).copyProperties(image)

def image_band_mean(imageCollection, scale, band):
    
    chart_ts_region = chart.Image.series(**{
    'imageCollection': imageCollection,
    'reducer': ee.Reducer.mean(),
    'region': aoi,
    'scale': scale,
    'band': band+'_mean',
    })
    
    return chart_ts_region.dataframe

def image_band_median(imageCollection, scale, band):
    
    chart_ts_region = chart.Image.series(**{
    'imageCollection': imageCollection,
    'reducer': ee.Reducer.median(),
    'region': aoi,
    'scale': scale,
    'band': band+'_median',
    })
    
    return chart_ts_region.dataframe

def image_band_percentile_5(imageCollection, scale, band):
    
    chart_ts_region = chart.Image.series(**{
    'imageCollection': imageCollection,
    'reducer': ee.Reducer.percentile([5]),
    'region': aoi,
    'scale': scale,
    'band': band+'_p5',
    })
    
    return chart_ts_region.dataframe

def image_band_percentile_95(imageCollection, scale, band):
    
    chart_ts_region = chart.Image.series(**{
    'imageCollection': imageCollection,
    'reducer': ee.Reducer.percentile([95]),
    'region': aoi,
    'scale': scale,
    'band': band+'_p95',
    })
    
    return chart_ts_region.dataframe

    
def bands_reduced_toexcel(imcol, scale, ind, band):
    '''
    Takes the bands and returns excel sheets of each catchment:
    ->mean, median, percentile, 
    '''
    df_mean = image_band_mean(imcol, scale, band)
    df_median = image_band_median(imcol, scale, band)
    df_pct5 = image_band_percentile_5(imcol, scale, band)
    df_pct95 = image_band_percentile_95(imcol, scale, band)
    
    
    df_mean.reset_index()
    df_median.reset_index()
    df_pct5.reset_index()
    df_pct95.reset_index()
    
    
    joined= df_mean.join(df_median, how='inner', lsuffix='mean', rsuffix='median')
    joined_pct = df_pct5.join(df_pct95, how='inner', lsuffix='p5', rsuffix='p95')
   
    annual = joined.join(joined_pct, how='inner')
    annual.to_excel(f'../Outputs/{dataset}/SR_timeseries/{ind}_annual_{band}.xlsx')

def extractArea(item):
    
    '''
    Method borrowed from https://code.earthengine.google.co.in/9c45ff677c46eae08952831de02bfb40
    Article: https://spatialthoughts.com/2020/06/19/calculating-area-gee/
    '''
    
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('classification')).format()
    area = ee.Number(areaDict.get('sum')).divide(1e6)
    return ee.List([classNumber, area])

def classArea(classified_image, scale):
    '''
    This function takes the pixel areas represented by each class the landsat scale is 30m but,
    nominal scale of image is 111000m after medoid compositing
    '''
    
    areaImage = ee.Image.pixelArea().addBands(classified_image)
    
    areas = areaImage.reduceRegion(**{
            'reducer':ee.Reducer.sum().group(**{'groupField':1, 'groupName':'classification'}),
            'geometry':aoi,
            'scale':scale,
            'maxPixels':1e10,
            'bestEffort':True,
    })
    
    classAreas = ee.List(areas.get('groups'))
    
    classAreasLists = classAreas.map(extractArea)
    
    return classAreasLists

def msToDate(milliseconds):
    base_datetime = datetime.datetime(1970, 1, 1)
    delta = datetime.timedelta(0, 0, 0, milliseconds)
    target_datetime = base_datetime + delta
    return target_datetime

def dataframeAreas(i, yc, classified, trainingClassImage, ms, classImageYear, name, accuracy, pixArea):

    ls1 = pd.DataFrame(classArea(classified, 30).getInfo(), columns=['class', 'area_RF'])
    ls2 = pd.DataFrame(classArea(trainingClassImage, 100).getInfo(), columns=['class', 'area_CORINE'])

    merged = ls1.merge(ls2, how='inner', on='class')
    merged['image_date'] = ms
    pivoted = merged.pivot(index='image_date', columns='class', values=['area_CORINE', 'area_RF'])
    pivoted['training', 'year_trained'] =  classImageYear
    pivoted['area_CORINE', '6'] = 0
    pivoted['catchment', 'area'] = pixArea
    pivoted['area_RF', '6'] = pivoted.catchment.area - pivoted.iloc[0, 6:10].sum() 
    pivoted['catchment', 'name '] = name
    pivoted['testing', 'accuracy'] = accuracy
    pivoted['ind'] = str(i)+'_'+str(yc)
    pivoted.fillna(0)
    
    return pivoted

def normalize (image):
    '''
    This function is used to convert band values to a range between 0 and 1 via normalisation,
    this is typically slow and no improvement to accuracy has been observed yet by its implementation.
    
    The 5 minute loop for an example catchment e.g. Chooz 2012, goes from 
    '''
    bandNames = image.bandNames()
    
def saveClassifierToCSV(classifier, name, yc):
    decisionTrees = ee.List(classifier.explain().get('trees')).getInfo()
    folder='Trees'

    var = f'../Outputs/{dataset}/{folder}/'

    if not os.path.exists(var):
        print('created')
        os.makedirs(var)

    ml.trees_to_csv(decisionTrees, f'../Outputs/Meuse/Trees/{name}_{yc}')


## Running Module over the Shapefile

1. The geometries are called by their system indices (sys_index) updating the 'aoi' and running the process over any  using the indices included in the 
2. The image collection is generated per shapefile and then returns the decadal mean of each index

# TODO:

- Redefine the methodology of reduction. Using chart --> dataframe --> join all dataframes is redundant an probably very slow

In [11]:

id_ls = 'used_images'
SR_t = 'SR_timeseries'
RF_c = 'RF_classification/'


folder_list = [id_ls, SR_t, RF_c]

for folder in folder_list:
    
    var = f'../Outputs/{dataset}/{folder}'
    
    if not os.path.exists(var):
        print('created')
        os.makedirs(var)

c_1990 = ml.csv_to_classifier('../Outputs/Meuse/Trees/Chooz_1990')
c_2000 = ml.csv_to_classifier('../Outputs/Meuse/Trees/Chooz_2000')
c_2006 = ml.csv_to_classifier('../Outputs/Meuse/Trees/Chooz_2006')
c_2012 = ml.csv_to_classifier('../Outputs/Meuse/Trees/Chooz_2012')
c_2018 = ml.csv_to_classifier('../Outputs/Meuse/Trees/Chooz_2018')


In [12]:
'''
Tuning of hyperParameters 'rfParams' is accomplished using this link:

https://code.earthengine.google.com/a1b6b96f28dc3c8998dcc74962b0eb51

JSON printed in the console was plotted in python to find the optimal parameters for 
this purpose. Specifically two years were plotted for the Chooz catchment 1990 vs 2018.

The optimal parameter for each year was identified using overall accuaracy and overall kappa score
Optimal parameter set for all years was determined to be between each.

The parameters are hard coded into the JS module:

Number of trees = 190
Varibles per split = 9
Minimum leaf population = 18
Bag fraction =  0.7
Max nodes = 400
Seed = 0

This results in an accuracy ranging from 84-86% max over the years

'''

classLoopParams = {'dataset':'CORINE', 
               'trainingClassLevel':1,
               'customClassLevels':None,
               'numClasses':5,            #if trainingClassLevel is 1 then there are 5 classes, level is 2 then there are 15, 3 is 44. (CORINE land cover class grouping)
               'split':0.7,               #split the training and testing 0.7/0.3 (70% training, 30% accuracy testing). 
               'tileScale':10,            #tileScale higher number reduces likelihood of classifier running into a memory limit
              }

scale = 5000 # define the pixel size for reducing, in meters, initially high to reduce comp time

In [ ]:
t0 = dt.today()

classArea_df = pd.DataFrame()

print(f'begin loop: {t0}')

for i, ind in enumerate(sys_index[1:]):

    
    name = df1.loc[ind].ID
    
    area = df1.loc[ind].area_km2
    
    pix_area = df1.loc[ind].pixel_area
    
    t1 = dt.today()
    
    print(f'\n{i+1}/{len(sys_index)} {t1}\nDataset: {dataset}, \nCatchment: {name}, \nSurface Reflectance Processing ...\n')
    
    aoi = Filtered_Sorted.filter(ee.Filter.eq('system:index', ind))
    
    annual_med = ltgee.buildSRcollection(startYear, endYear, startDay, endDay, aoi, maskThese)
    
    annual_med_calc = ltgee.transformSRcollection(annual_med, bandList)
    print('saving bands to excel')
    band_calc_df = bands_reduced_toexcel(ee.ImageCollection(annual_med_calc), scale, ind, 'B1')
    
    print('calculating LAI')
    
    col = ee.ImageCollection(ltgee.LAIcol(startYear, endYear, startDay, endDay, aoi)) 
    
    lai_calc_df = bands_reduced_toexcel(col, col.first().projection().nominalScale().getInfo(), ind, 'LAI')
    
    '''
    
    Which images are used in creating the annual composites?
    -> return as a list of Landsat image IDs ** 
    ** can be used for exclusion if image quality is suboptimal upon later inspection.. important for small sample cases.
    
    '''
    
    id_key = 'idList'
    
    masked_col_key = 'collection'
    
    print('getting ID collection')
    
#     GetCollectionID = ltgee.getCollectionIDlist(startYear, endYear, startDay, endDay, aoi)
    
#     im_id_list = GetCollectionID[id_key]
    
#     image_list = pd.DataFrame({f'{name}': im_id_list.getInfo()})
    
#     image_list.to_csv(f'../Outputs/{dataset}/used_images/{ind}_imageList.csv')
    
    
    t2 = dt.today()
    
    print(f'step 1: Surface reflectance exported: {t2} \nTime taken: {t2-t1}')
    
    t3 = dt.today()
    
    print(f'\nstep 2: Initialize classification routine: {t3}')
    
    
    if dataset == 'CAMELS_GB':
        '''
        
        The years that we train on are not necessarily the same as the years we classify: 
        - first define the years to return that will be relevant to the decadal analysis (matching the hydroclimatic decades)
        - Then use conditions to define which training set corresponds best to the image to be classified. 
        - LATER: Will need to use a trained classifier, perhaps the 'best performer', to classify the USA dataset. 
            - Best accuracy may be to take a classifier that samples all 4 categories
        
        '''
        # year_classified = np.arange(1984, 2018)
        year_classified = [1986,1985,1986, 1994,1995,1996,2004,2005,2006,2014,2015,2016]
        
        for j, yc in enumerate(year_classified):
            '''
            Define the training image, then the image to classify, adding slope and elevation bands

            Corine Representative Classes GB:
            || 1989 -> 1998 | 1999 -> 2001 | 2005 -> 2007 | 2011 -> 2012 | 2017 -> 2018 ||
            ||   "2000   |    "2000"    |    "2006"    |     "2012"   |     "2018"   ||

            Training "image year above" --> classify the relevant Landsat date range below:
            || 1984 -> 1998 | 1999 -> 2003 | 2004 -> 2009 | 2010 -> 2014 | 2015 -> ...  || 
            '''
            if yc >= 1984 and yc < 1999:
                classifier = c_1990
                classImageYear = 2000

            elif yc >= 1999 and yc < 2004:
                classifier = c_2000
                classImageYear = 2000

            elif yc >= 2004 and yc < 2010:
                classifier = c_2006
                classImageYear = 2006

            elif yc >= 2010 and yc < 2015:
                classifier = c_2012
                classImageYear = 2012

            elif yc >= 2015:
                classifier = c_2018
                classImageYear = 2018

            else:
                print('ERROR: year to classify out of range[1984 - 2022]')
                break
            
            #the image from the collection that we want to classify
            imageFromCollection = ee.ImageCollection(annual_med_calc).filterDate(str(yc)+'-'+startDay, str(yc+1)+'-'+endDay).first().clip(aoi)
            
            #image from training dataset e.g. CORINE is selected and simlified... 
            trainingClassImage = ltgee.createTrainingImage(str(classImageYear), classLoopParams['dataset'], classLoopParams['trainingClassLevel'], aoi)
            
            
            #Adding the elevation and slope band calculations to each image
            imageToClassify = ltgee.addTerrainBand(imageFromCollection, aoi)
            
            #getting the date of the image and converting it from milliseconds since 1970 (Earth engines preferred datetime)
            ms = msToDate(ee.Date(imageToClassify.get('system:time_start')).getInfo()['value'])
            
            #the points used for training the classifier are randomly distibuted amongst the classes extracting a profile of spectral and terrain
            points = ltgee.genGCP(trainingClassImage, imageToClassify, classLoopParams['numClasses'], classLoopParams['split'], classLoopParams['tileScale'], aoi, 'weighted')
            
            # 70% of the points are allocated to training
            training = points['training']
            
            #30% of the points are allocated to classification
            testing = points['testing']
            
            t5 = dt.today()
            
            # classifier training with predefined number of trees using training points
            # classifier = ltgee.classifier(imageToClassify, training)
            
            t6 = dt.today()
            
            #saving decision trees for later use
            # saveClassifierToCSV(classifier, name, yc)
            
#             #classifying image using the training
            classified = imageToClassify.classify(classifier)
            
            focal = classified.focalMode(**{'radius':30,
                                          'kernelType':'square',
                                          'units':'meters',
                                          'iterations':2}).clip(aoi)
            
            # Map.addLayer(focal, {'bands':['classification'], 'min':1, 'max':5, 'palette':['#E6004D', '#FFFFA8', '#80FF00', '#A6A6FF', '#00CCF2']}, f'{name} RF:{j}.{yc}.{classImageYear}')

#           #assess the accuracy using the testing points, see where the confusion occurs
            accuracy = testing.classify(classifier).errorMatrix('landcover', 'classification').accuracy().getInfo()
            print(f'{yc} classified using: {classImageYear} ...  \naccuracy: {accuracy:.3f}')


            df = dataframeAreas(i, j, classified, trainingClassImage, ms, classImageYear, name, accuracy, pix_area)
            
            df.to_csv(f'../Outputs/{dataset}/RF_classification/{ind}_{yc}_classes.csv')
            
            classArea_df = classArea_df.append(df)
            
    else:
        print('classification routine for this dataset is not yet provided for')
    
    t4 = dt.today()
    
    print(f'step2: Done: {t4}, time taken: {t4-t3}')
    
    print(f'\nCatchment: {name}, total time: {t4-t1}\n---------------')
    
#     if ind == sys_index[0]:
#         break



tfinal = dt.today()

print(f'END LOOP: Full routine finished: {tfinal} \nTime taken: {tfinal-t0}')

begin loop: 2023-04-01 20:45:21.165099

1/95 2023-04-01 20:45:21.168091
Dataset: CAMELS_GB, 
Catchment: 71001, 
Surface Reflectance Processing ...

saving bands to excel
calculating LAI
getting ID collection
step 1: Surface reflectance exported: 2023-04-01 20:47:37.219646 
Time taken: 0:02:16.051555

step 2: Initialize classification routine: 2023-04-01 20:47:37.219646
1986 classified using: 2000 ...  
accuracy: 0.385
1985 classified using: 2000 ...  
accuracy: 0.582
1986 classified using: 2000 ...  
accuracy: 0.385
1994 classified using: 2000 ...  
accuracy: 0.478
1995 classified using: 2000 ...  
accuracy: 0.697
1996 classified using: 2000 ...  
accuracy: 0.516
2004 classified using: 2006 ...  
accuracy: 0.440
2005 classified using: 2006 ...  
accuracy: 0.561
2006 classified using: 2006 ...  
accuracy: 0.451
2014 classified using: 2012 ...  
accuracy: 0.449
2015 classified using: 2018 ...  
accuracy: 0.634
2016 classified using: 2018 ...  
accuracy: 0.631
step2: Done: 2023-04-01 21:2

## Trend analysis per band

- can we look at median band collection and percentile bounds, reduced to a per kilometer scale

In [ ]:
train_vis = {'bands':['landcover'], 'min':1, 'max':5, 'palette':['#E6004D', '#FFFFA8', '#80FF00', '#A6A6FF', '#00CCF2']}

class_vis = {'bands':['classification'], 'min':1, 'max':5, 'palette':['#E6004D', '#FFFFA8', '#80FF00', '#A6A6FF', '#00CCF2']}

Map = geemap.Map()
Map.centerObject(aoi)
Map.addLayer(aoi)
Map.addLayer(imageToClassify, RGB_VIS, 'RGB')
Map.addLayer(trainingClassImage, train_vis, 'Train')
Map.addLayer(classified, class_vis, 'Class')
Map